In [2]:
"""Full dataframe logger from xml file"""



def pull_marker_data(xml_file):
    import xml.etree.ElementTree as et
    import pandas as pd
    file = et.parse(xml_file)
    root = file.getroot()
    marker_data = []

    MDElement = root.find('Marker_Data')
    for Marker_Type in MDElement.findall('Marker_Type'):
        type_id = int(Marker_Type.find('Type').text)

        for Marker in Marker_Type.findall('Marker'):
            xloc = int(Marker.find('MarkerX').text) if Marker.find('MarkerX') is not None else None
            xloc = int(Marker.find('MarkerX').text) - 1 if Marker.find('MarkerX') is not None else None
            yloc = int(Marker.find('MarkerY').text) if Marker.find('MarkerY') is not None else None
            yloc = int(Marker.find('MarkerY').text) - 1 if Marker.find('MarkerY') is not None else None
            zloc = int(Marker.find('MarkerZ').text) if Marker.find('MarkerZ') is not None else None
            zloc = int(Marker.find('MarkerZ').text) - 1 if Marker.find('MarkerZ') is not None else None
            zlayer = (2 + zloc) // 3
            zlayer = (2 + zloc) // 3  # Keep this logic as is

            marker_data.append({
                'z': zlayer,
                'type_id': type_id,
                'y': yloc,
                'x': xloc,
            })

    dataframe = pd.DataFrame(marker_data, columns=['z', 'type_id', 'y', 'x'])
    return dataframe




FileNotFoundError: [Errno 2] No such file or directory: 'FF15A/CellCounter_FF151L2L.xml'

In [3]:
""" create the most distinct colors from the dataframe"""



def find_distinct_colors(xml_file):
    import numpy as np
    import colorsys

    df = pull_marker_data(xml_file)
    num_colors = max(df['type_id'])

    colors = []
    for i in range(num_colors):
        r, g, b = colorsys.hsv_to_rgb(i / num_colors, 1.0, 1.0)
        rgb = np.array([int(r * 255), int(g * 255), int(b * 255)])
        colors.append(rgb)

    return colors





FileNotFoundError: [Errno 2] No such file or directory: 'FF15A/CellCounter_FF151L2L.xml'

In [5]:
"""full 3 channel hue color tiff writer from dataframe log"""

"full working code readable on napari image viewer"

def pull_synapse_and_mark_file(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    df = pull_marker_data(xml_file)
    color_array = find_distinct_colors(xml_file)
    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    rgb_image = np.zeros((layers, 3, height, width), dtype=np.uint8)
    x = df['x']
    y = df['y']
    z = df['z']
    type_id = df['type_id']
    for i in range(len(df)):
        z_idx = max(0, min(z[i], layers - 1))  # Ensure z index is in range
        y_idx = max(0, min(y[i], height - 1))  # Ensure y index is in range
        x_idx = max(0, min(x[i], width - 1))   # Ensure x index is in range
        rgb_image[z_idx, :, y_idx, x_idx] = color_array[type_id[i] - 1]
    tifffile.imwrite(output_file, rgb_image)
    return rgb_image

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
  

In [ ]:
"""automated rgb image marker from folder"""

def mark_folder(file_folder):
    import os
    import numpy as np
    import tifffile

    folder = file_folder
    output = folder + "_markers"
    os.makedirs(output, exist_ok=True)
    lsm_list = [file for file in os.listdir(folder) if file.endswith(".lsm") and not (file.startswith(".") or file.endswith("old.lsm"))]
    if len(lsm_list) == 0:
        print("No .lsm files found in the folder.")
        return
    else:
        for file in lsm_list:
            print(file)
            image = tifffile.imread(folder + "/" + file)
            layers, channels, height, width = image.shape
            rgb_image = np.zeros((layers, channels, height, width), dtype=np.uint8)
            xml_file_path = folder + "A/CellCounter_" + file.replace(".lsm", ".xml")
            xml_alternate_path = folder + "A_/CellCounter_" + file.replace(".lsm", ".xml")
            xml_window_path = folder + "A/CellCounter_Counter Window - " + file.replace(".lsm", ".xml")
            if os.path.exists(xml_file_path):
                df = pull_marker_data(xml_file_path)
                color_array = find_distinct_colors(xml_file_path)
            else:
                if os.path.exists(xml_alternate_path):
                    df = pull_marker_data(xml_alternate_path)
                    color_array = find_distinct_colors(xml_alternate_path)
                else:
                    if os.path.exists(xml_window_path):
                        df = pull_marker_data(xml_window_path)
                        color_array = find_distinct_colors(xml_window_path)
                    else:
                        if os.path.exists(folder + "Counter Window - " + file.replace(".lsm", ".xml")):

                            df = pull_marker_data(folder + "Counter Window - " + file.replace(".lsm", ".xml"))
                            color_array = find_distinct_colors(folder + "Counter Window - " + file.replace(".lsm", ".xml"))

                        else:
                            print(xml_file_path + "does not exist. Skipping file.")

            x = df['x']
            y = df['y']
            z = df['z']
            type_id = df['type_id']
            filename = file.replace(".lsm", "_marked.tif")
            filepath = os.path.join(output, filename)
            for i in range(len(df)):
                rgb_image[z[i]-1, :, y[i]-1, x[i]-1] = color_array[type_id[i]-1]
            tifffile.imwrite(filepath, rgb_image)


In [13]:
"""Automated 255 greyscale marker from folder"""


def mark_folder255(file_folder):
    import os
    import numpy as np
    import tifffile

    folder = file_folder
    output = folder + "_markers"
    os.makedirs(output, exist_ok=True)
    lsm_list = [file for file in os.listdir(folder) if file.endswith(".lsm") and not (file.startswith(".") or file.endswith("old.lsm"))]
    if len(lsm_list) == 0:
        print("No .lsm files found in the folder.")
        return
    else:
        for file in lsm_list:
            print(file)
            image = tifffile.imread(folder + "/" + file)
            layers, channels, height, width = image.shape
            rgb_image = np.zeros((layers, channels, height, width), dtype=np.uint8)
            xml_file_path = folder + "A/CellCounter_" + file.replace(".lsm", ".xml")
            xml_alternate_path = folder + "A_/CellCounter_" + file.replace(".lsm", ".xml")
            xml_window_path = folder + "A/CellCounter_Counter Window - " + file.replace(".lsm", ".xml")
            if os.path.exists(xml_file_path):
                df = pull_marker_data(xml_file_path)
            else:
                if os.path.exists(xml_alternate_path):
                    df = pull_marker_data(xml_alternate_path)
                else:
                    if os.path.exists(xml_window_path):
                        df = pull_marker_data(xml_window_path)
                    else:
                        if os.path.exists(folder + "Counter Window - " + file.replace(".lsm", ".xml")):

                            df = pull_marker_data(folder + "Counter Window - " + file.replace(".lsm", ".xml"))

                        else:
                            print(xml_file_path + "does not exist. Skipping file.")
                            continue  # Skip to the next file if XML does not exist

            x = df['x']
            y = df['y']
            z = df['z']
            type_id = df['type_id']
            filename = file.replace(".lsm", "_marked.tif")
            filepath = os.path.join(output, filename)
            color_array = [255, 255, 255]
            for i in range(len(df)):
                z_idx = max(0, min(z[i] - 1, layers - 1))  # Ensure z index is in range
                y_idx = max(0, min(y[i] - 1, height - 1))  # Ensure y index is in range
                x_idx = max(0, min(x[i] - 1, width - 1))   # Ensure x index is in range
                rgb_image[z_idx, :, y_idx, x_idx] = color_array
            tifffile.imwrite(filepath, rgb_image)

In [14]:
import os
path = r"C:\Users\richa\parralabimageprocessing\Synapse Confocal Images Rat"
for folder in os.listdir(path):
    if not (folder.startswith(".") or folder.startswith("w")):
        mark_folder255(os.path.join(path, folder))


BAT1L2L.lsm
BAT1L2R.lsm
BAT1L5L.lsm
BAT1L5R.lsm
BAT2L2L.lsm
BAT2L2R.lsm
BAT2L5L.lsm
BAT2L5R.lsm
No .lsm files found in the folder.
FF151L2L.lsm
FF151L2R.lsm
FF151L5L.lsm
FF151L5R.lsm
FF152L2L.lsm
FF152L2R.lsm
FF152L5L.lsm
FF152L5R.lsm
No .lsm files found in the folder.
FF171L2L.lsm
FF171L2R.lsm
FF171L5L.lsm
FF171L5R.lsm
FF172L2L.lsm
FF172L2R.lsm
FF172L5L.lsm
FF172L5R.lsm
No .lsm files found in the folder.
FF231L2L.lsm
FF231L2R.lsm
FF231L5L.lsm
FF231L5R.lsm
FF232L2L.lsm
FF232L2R.lsm
FF232L5L.lsm
FF232L5R.lsm
No .lsm files found in the folder.
OBFF091L2L.lsm
OBFF091L2R.lsm
OBFF091L5L.lsm
OBFF091L5R.lsm
OBFF092L2L.lsm
OBFF092L2R.lsm
OBFF092L5L.lsm
OBFF092L5R.lsm
No .lsm files found in the folder.
OBFF11L2L.lsm
OBFF11L2R.lsm
OBFF11L5L.lsm
OBFF11L5R.lsm
OBFF12L2L.lsm
OBFF12L2R.lsm
OBFF12L5L.lsm
OBFF12L5R.lsm
OBFF101L2L.lsm
OBFF101L2R.lsm
OBFF101L5L.lsm
OBFF101L5R.lsm
OBFF102L2L.lsm
OBFF102L2R.lsm
OBFF102L5L.lsm
OBFF102L5R.lsm
No .lsm files found in the folder.
OBFF111L2L.lsm
OBFF111L2R.lsm


In [15]:
pull_synapse_and_mark_file(r'C:\Users\richa\parralabimageprocessing\Synapse Confocal Images Rat\OBFF4A\Counter Window - OBFF42L5L.xml', r'C:\Users\richa\parralabimageprocessing\Synapse Confocal Images Rat\OBFF4\OBFF42L5L.lsm', r'C:\Users\richa\parralabimageprocessing\Synapse Confocal Images Rat\OBFF4_markers\OBFF42L5L_marked.tif')

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
  